<a href="https://colab.research.google.com/github/Abhinaba09/Handwritten-Signature-Verification-using-Longitudinal-Data/blob/main/Handwritten_Signature_Verification_using_Writer_Dependent_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create directory structure and dummy images
def create_directory_structure():
    base_path = 'signature_data'
    writers = ['writer1', 'writer2']
    subdirs = ['genuine', 'forged']

    for writer in writers:
        for subdir in subdirs:
            path = os.path.join(base_path, writer, subdir)
            os.makedirs(path, exist_ok=True)

    # Function to create dummy images
    def create_dummy_image(path, num_images):
        for i in range(num_images):
            img = np.random.randint(0, 256, (100, 200), dtype=np.uint8)  # Dummy grayscale image
            filename = os.path.join(path, f'signature_{i+1}.png')
            cv2.imwrite(filename, img)

    # Generate dummy images for each writer and subdirectory
    num_images = 10
    for writer in writers:
        for subdir in subdirs:
            path = os.path.join(base_path, writer, subdir)
            create_dummy_image(path, num_images)

    print("Directory structure and dummy images created.")

# Step 1: Load Data
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
    return images

def load_data(base_path):
    data = []
    labels = []
    writers = os.listdir(base_path)
    for writer in writers:
        genuine_path = os.path.join(base_path, writer, 'genuine')
        forged_path = os.path.join(base_path, writer, 'forged')

        genuine_images = load_images_from_folder(genuine_path)
        forged_images = load_images_from_folder(forged_path)

        for img in genuine_images:
            data.append(img)
            labels.append(1)  # Genuine signature

        for img in forged_images:
            data.append(img)
            labels.append(0)  # Forged signature

    return data, labels

# Step 2: Preprocess Signatures
def preprocess_image(img):
    # Resize to a fixed size
    img = cv2.resize(img, (128, 128))
    # Normalize
    img = img / 255.0
    return img

# Step 3: Feature Extraction
def extract_features(images):
    features = []
    for img in images:
        img = preprocess_image(img)
        # Extract HOG features
        feature_vector, _ = hog(img, orientations=8, pixels_per_cell=(16, 16),
                                cells_per_block=(1, 1), visualize=True)
        features.append(feature_vector)
    return features

# Step 4: Train a Model
def train_model(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    model = svm.SVC(kernel='linear', probability=True)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    return model

# Step 5: Verify Signatures
def verify_signature(model, img):
    img = preprocess_image(img)
    feature_vector, _ = hog(img, orientations=8, pixels_per_cell=(16, 16),
                            cells_per_block=(1, 1), visualize=True)
    feature_vector = feature_vector.reshape(1, -1)
    prediction = model.predict(feature_vector)
    probability = model.predict_proba(feature_vector)
    return prediction, probability

# Create directory structure and dummy images
create_directory_structure()

# Load data
base_path = 'signature_data'
data, labels = load_data(base_path)

# Extract features
features = extract_features(data)

# Train model
model = train_model(features, labels)

# Verify a new signature
test_image_path = os.path.join(base_path, 'writer1', 'genuine', 'signature.jpg')
test_image = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
prediction, probability = verify_signature(model, test_image)

print("Prediction (1 = Genuine, 0 = Forged):", prediction[0])
print("Probability:", probability)


Directory structure and dummy images created.
Accuracy: 0.375


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
